In [45]:
from sklearn.cluster import DBSCAN
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import pandas as pd
from open3d import visualization, io
import matplotlib.pyplot as plt
import ast
import os

In [46]:
color_map = {
    0: [1, 0, 0],     # Cluster 0 - Red
    1: [0, 1, 0],     # Cluster 1 - Green
    2: [0, 0, 1],     # Cluster 2 - Blue
    3: [1, 1, 0],     # Cluster 3 - Yellow
    4: [1, 0, 1],     # Cluster 4 - Magenta
    5: [0, 1, 1],     # Cluster 5 - Cyan
}

In [47]:
def dbscan_clustering(df: pd.DataFrame, threshold_motion= 0.05, eps= 1.0, min_samples= 10):
    # Points with motion
    mask_motion = df.loc[:, "avg_motion_vector_mag"] >= threshold_motion
    
    points_motion = np.vstack(df[mask_motion].loc[:, "source_point"].values)
    avg_motion_vector = np.vstack(df[mask_motion].loc[:, "avg_motion_vector"].values)
    features = points_motion
    # features = np.hstack((points_motion, avg_motion_vector))
    
    # DBSCAN only for points, that were with motion-detected marked
    clustering = DBSCAN(eps= eps, min_samples= min_samples).fit(features)
    labels = clustering.labels_
    
    num_clusters = len(set(labels)) - (1 if -1 in labels else 0)    # noise points ignored
    num_noise = np.sum(labels == -1)
    print(f"{num_clusters} clusters found")
    # print(f"Points of noise: {num_noise}")
    
    # Find the cluster with the highest degree of aggregation (the cluster with the most points)
    unique_labels = set(labels)
    label_counts = {}
    
    for label in unique_labels:
        label_counts[label] = np.sum(labels == label)
    
    # for label, count in label_counts.items():
    #     print(f"Cluster {label} contains: {count} points")

    max_cluster_label = max(label_counts, key= label_counts.get)
    
    
    # Points without motion
    mask_no_motion = df.loc[:, "avg_motion_vector_mag"] < threshold_motion
    points_no_motion = np.vstack(df[mask_no_motion].loc[:, "source_point"].values)
    
    
    # Assign colors for different clusters
    unique_labels = set(labels)
    colors_points_motion = []
    
    for label in labels:
        if label == -1:
            colors_points_motion.append([0.5, 0.5, 0.5])  # gray for noise points
        else:
            colors_points_motion.append(color_map.get(label, [0.5, 0.5, 0.5]))
    
    ## for points_no_motion
    colors_points_no_motion = []
    for i in range(points_no_motion.shape[0]):
        colors_points_no_motion.append([0.5, 0.5, 0.5])  # gray
    
    
    # Visualization
    cloud_points_motion = o3d.geometry.PointCloud()
    cloud_points_motion.points = o3d.utility.Vector3dVector(points_motion)
    cloud_points_motion.colors = o3d.utility.Vector3dVector(colors_points_motion)
    
    cloud_points_no_motion = o3d.geometry.PointCloud()
    cloud_points_no_motion.points = o3d.utility.Vector3dVector(points_no_motion)
    cloud_points_no_motion.colors = o3d.utility.Vector3dVector(colors_points_no_motion)
    
    return cloud_points_motion + cloud_points_no_motion

In [51]:
Path_dfs = "./dfs/sliding_windowsize_5"
Path_pcds_processed = "./pcds_processed"
for count_df in range(0, 54):
    df = pd.read_pickle(os.path.join(Path_dfs, "df_{:0>3}.pkl".format(count_df)))
    point_cloud = dbscan_clustering(df, eps= 0.4, min_samples= 25, threshold_motion= 0.05)
    o3d.io.write_point_cloud(os.path.join(Path_pcds_processed, "point_cloud_{:0>3}.pcd".format(count_df)), point_cloud)

1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found


In [52]:
num_example_point_cloud = 30
example_point_cloud = io.read_point_cloud(os.path.join(Path_pcds_processed, "point_cloud_{:0>3}.pcd".format(num_example_point_cloud))) # Read point cloud
o3d.visualization.draw_geometries([example_point_cloud])